In [1]:
import pandas as pd
from datetime import datetime, timedelta
import geopandas as gpd

In [4]:
spatial = gpd.read_file('../Data/SHP_MGN2018_INTGRD_MPIO/MGN_ANM_MPIOS.shp')
spatial1 = spatial.copy()
spatial1['MPIO_CDPMP'] = pd.to_numeric(spatial1['MPIO_CDPMP'])
spatial1 = spatial1[['MPIO_CDPMP', 'AREA', 'LATITUD', 'LONGITUD']]
spatial1

,MPIO_CDPMP,AREA,LATITUD,LONGITUD
0,18001,2.547638e+09,1.749139,-75.558239
1,18029,4.141221e+08,1.227865,-75.882327
2,18094,1.191619e+09,1.500923,-75.875645
3,18247,1.106076e+09,1.791386,-75.193944
4,18256,1.234734e+09,1.617746,-75.234043
...,...,...,...,...
1117,25290,1.939529e+08,4.323534,-74.388591
1118,25662,3.140873e+08,4.841098,-74.658493
1119,73349,3.048869e+08,5.179417,-74.783031
1120,5631,1.583532e+07,6.138305,-75.608806


In [5]:
fechas_semanales = pd.date_range(start='2007-01-01', end='2021-12-31', freq='W')

# Replicar los datos para cada semana
num_semanas = len(fechas_semanales)
spatial_replicado = pd.concat([spatial1] * num_semanas, ignore_index=True)

spatial_replicado = spatial_replicado.sort_values(['MPIO_CDPMP'])
# Asignar las fechas al DataFrame replicado
spatial_replicado['Date'] = sorted(fechas_semanales) * len(spatial1)

# Restablecer el índice si es necesario
spatial_replicado.reset_index(drop=True, inplace=True)

# Mostrar el DataFrame resultante
print(spatial_replicado)

        MPIO_CDPMP          AREA   LATITUD   LONGITUD       Date
0             5001  3.748306e+08  6.257590 -75.611031 2007-01-07
1             5001  3.748306e+08  6.257590 -75.611031 2007-01-14
2             5001  3.748306e+08  6.257590 -75.611031 2007-01-21
3             5001  3.748306e+08  6.257590 -75.611031 2007-01-28
4             5001  3.748306e+08  6.257590 -75.611031 2007-02-04
...            ...           ...       ...        ...        ...
877399       99773  6.559721e+10  4.262455 -69.521405 2021-11-28
877400       99773  6.559721e+10  4.262455 -69.521405 2021-12-05
877401       99773  6.559721e+10  4.262455 -69.521405 2021-12-12
877402       99773  6.559721e+10  4.262455 -69.521405 2021-12-19
877403       99773  6.559721e+10  4.262455 -69.521405 2021-12-26

[877404 rows x 5 columns]


In [6]:
cities = spatial1['MPIO_CDPMP'].unique()

In [7]:
df = pd.read_csv('../Data/dengue_mpios.csv')
df

,COD_EVE,COD_MUN_O,FEC_NOT,CASES
0,210,1032,2007-01-01,0
1,210,1036,2007-01-01,0
2,210,1050,2007-01-01,0
3,210,1068,2007-01-01,0
4,210,1076,2007-01-01,0
...,...,...,...,...
2621479,580,99001,2021-12-26,0
2621480,580,99524,2021-12-26,0
2621481,580,99624,2021-12-26,0
2621482,580,99773,2021-12-26,0


In [8]:
df = df.loc[df['COD_MUN_O'].isin(cities)]
df['FEC_NOT'] = pd.to_datetime(df['FEC_NOT'])
df['FEC_NOT'] = df['FEC_NOT'] + pd.DateOffset(weekday=6)

df_pivot = pd.pivot_table(df, values='CASES', index=['FEC_NOT', 'COD_MUN_O'], columns='COD_EVE').reset_index()

dengue = df_pivot.fillna(0).reset_index(drop=True)

dengue.columns = ["Date", "Mpio", "Dengue", "Dengue_grave", "Mortalidad_por_dengue"]

print(dengue)

C:\Users\Juan\AppData\Local\Temp\ipykernel_12196\85166581.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['FEC_NOT'] = pd.to_datetime(df['FEC_NOT'])
C:\Users\Juan\AppData\Local\Temp\ipykernel_12196\85166581.py:5: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex.
  df['FEC_NOT'] = df['FEC_NOT'] + pd.DateOffset(weekday=6)
C:\Users\Juan\AppData\Local\Temp\ipykernel_12196\85166581.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['FEC_NOT'] = df['FEC_NOT'] + pd.

             Date   Mpio  Dengue  Dengue_grave  Mortalidad_por_dengue
0      2007-01-07   5001     0.0           0.0                    0.0
1      2007-01-07   5002     0.0           0.0                    0.0
2      2007-01-07   5004     0.0           0.0                    0.0
3      2007-01-07   5021     0.0           0.0                    0.0
4      2007-01-07   5030     0.0           0.0                    0.0
...           ...    ...     ...           ...                    ...
834389 2021-12-26  97889     0.0           0.0                    0.0
834390 2021-12-26  99001     0.0           0.0                    0.0
834391 2021-12-26  99524     0.0           0.0                    0.0
834392 2021-12-26  99624     0.0           0.0                    0.0
834393 2021-12-26  99773     1.0           0.0                    0.0

[834394 rows x 5 columns]


In [9]:
climate = pd.read_csv("../Data/climate_mpios.csv")
# climate = climate.loc[climate['mpio'].isin(cities)]
climate['id'] = pd.to_datetime(climate['id'])
climate['id'] = climate['id'] + pd.DateOffset(weekday=6)

climate = climate.drop_duplicates(subset = ['id', 'mpio'], keep='first')

fecha_inicio = '2007-01-01'
fecha_fin = '2021-12-31'
frecuencia = 'W'
rango_fechas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq=frecuencia)

codigos_departamento = climate['mpio'].unique()
combinaciones = [(codigo, fecha) for codigo in codigos_departamento for fecha in rango_fechas]
climate_combinado = pd.DataFrame(combinaciones, columns=['mpio', 'id'])

climate = pd.merge(climate_combinado, climate, on=['mpio', 'id'], how='left')

climate = climate.sort_values(by=['id'])

grupos = climate.groupby('mpio')
climate['max_temperature'] = grupos['max_temperature'].fillna(method='ffill')
climate['min_temperature'] = grupos['min_temperature'].fillna(method='ffill')
climate['mean_temperature'] = grupos['mean_temperature'].fillna(method='ffill')

climate = climate[['mpio', 'id', 'max_temperature', 'mean_temperature',
       'min_temperature']]

climate = climate.sort_values('mpio').fillna(method = 'ffill')
climate = climate.sort_values('id')
climate

C:\Users\Juan\AppData\Local\Temp\ipykernel_12196\3185275470.py:4: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex.
  climate['id'] = climate['id'] + pd.DateOffset(weekday=6)


,mpio,id,max_temperature,mean_temperature,min_temperature
674084,25875,2007-01-07,29.41,26.014663,22.63
333914,27495,2007-01-07,31.05,28.052597,18.97
394128,13433,2007-01-07,36.07,32.265613,28.77
455124,47545,2007-01-07,38.41,34.687196,28.45
696762,54553,2007-01-07,31.59,29.500128,28.19
...,...,...,...,...,...
122773,52323,2021-12-26,21.57,12.931046,6.97
819535,68296,2021-12-26,32.35,20.318984,9.17
580243,15776,2021-12-26,29.89,23.528353,14.01
289339,5591,2021-12-26,29.21,26.450109,21.03


In [10]:
rain = pd.read_csv('../Data/rain_mpios.csv')
rain

,id,mean_rain,mpio
0,2007-01-01,2.888476,18001
1,2007-01-06,3.640295,18001
2,2007-01-11,5.167350,18001
3,2007-01-16,15.311894,18001
4,2007-01-21,53.494459,18001
...,...,...,...
1211755,2021-12-06,4.568261,5380
1211756,2021-12-11,26.734908,5380
1211757,2021-12-16,14.426954,5380
1211758,2021-12-21,13.533269,5380


In [11]:
# rain = rain.loc[rain['mpio'].isin(cities)]
rain['id'] = pd.to_datetime(rain['id'])
rain['id'] = rain['id'] + pd.DateOffset(weekday=6)
rain = rain.drop_duplicates(subset = ['id', 'mpio'], keep='first')

fecha_inicio = '2007-01-01'
fecha_fin = '2021-12-31'
frecuencia = 'W'
rango_fechas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq=frecuencia)

codigos_departamento = rain['mpio'].unique()
combinaciones = [(codigo, fecha) for codigo in codigos_departamento for fecha in rango_fechas]
rain_combinado = pd.DataFrame(combinaciones, columns=['mpio', 'id'])

rain = pd.merge(rain_combinado, rain, on=['mpio', 'id'], how='left')

rain = rain.sort_values(by=['id'])

grupos = rain.groupby('mpio')
rain['mean_rain'] = grupos['mean_rain'].fillna(method='ffill')

rain

C:\Users\Juan\AppData\Local\Temp\ipykernel_12196\922664042.py:3: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex.
  rain['id'] = rain['id'] + pd.DateOffset(weekday=6)


,mpio,id,mean_rain
0,18001,2007-01-07,2.888476
31280,19573,2007-01-07,7.610504
560694,15599,2007-01-07,2.392269
125120,52227,2007-01-07,12.394630
797640,68397,2007-01-07,16.401631
...,...,...,...
584935,15808,2021-12-26,6.809893
585717,15810,2021-12-26,4.109981
586499,15814,2021-12-26,4.967791
581807,15790,2021-12-26,4.357695


In [12]:
pop = pd.read_csv('../Data/population_projection.csv', sep=';')
pop

,DPMP,ANO,Hombres_0,Hombres_1,Hombres_2,Hombres_3,Hombres_4,Hombres_5,Hombres_6,Hombres_7,...,Total_95,Total_96,Total_97,Total_98,Total_99,Total_100,Total_80+,Total_Hombres,Total_Mujeres,Total_General
0,5001,2007,13926,14328,14657,15071,15510,15918,16355,16841,...,366,287,234,196,159,477,32237,978964,1122807,2101771
1,5001,2008,13750,14123,14510,14839,15238,15633,16063,16523,...,393,311,250,212,170,510,33719,992558,1136778,2129336
2,5001,2009,13633,13971,14333,14710,15026,15376,15790,16240,...,415,334,271,229,184,548,35170,1006209,1150885,2157094
3,5001,2010,13608,13911,14231,14575,14929,15203,15569,15994,...,439,356,295,248,201,595,36681,1020564,1164975,2185539
4,5001,2011,13712,13972,14250,14542,14850,15156,15443,15807,...,465,377,316,269,218,644,38283,1035440,1178109,2213549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16825,99773,2017,1063,1051,1055,1056,1059,1064,1065,1063,...,5,4,5,3,1,16,380,37809,32530,70339
16826,99773,2018,1040,1048,1054,1063,1073,1075,1077,1075,...,5,3,2,1,3,25,372,38690,34001,72691
16827,99773,2019,1066,1048,1063,1073,1081,1091,1092,1091,...,2,3,3,3,0,24,398,40488,35708,76196
16828,99773,2020,1092,1062,1054,1071,1080,1088,1095,1095,...,6,1,3,3,3,18,420,41840,37023,78863


In [13]:
columns = ['DPMP', 'ANO', 'Hombres_0_19', 'Hombres_20_39', 'Hombres_40_59',
           'Hombres_60_79', 'Hombres_80+', 'Mujeres_0_19', 'Mujeres_20_39',
           'Mujeres_40_59', 'Mujeres_60_79', 'Mujeres_80+', 'Total_0_19',
           'Total_20_39', 'Total_40_59', 'Total_60_79', 'Total_80+',
           'Total_Hombres', 'Total_Mujeres', 'Total_General']
pop = pop[columns]
# pop = pop.loc[pop['DPMP'].isin(cities)]
pop_W = []
for año in [2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021]:

    fechas_semanales = pd.date_range(start=str(año)+'-01-01', end=str(año)+'-12-31', freq='W')
    
    df_final = pd.DataFrame()

    dptos = pop['DPMP'].unique()

    num_repeticiones = len(fechas_semanales)
    pop_s = pop[pop['ANO']==año]
    pop_w = pop_s.loc[pop_s.index.repeat(num_repeticiones)]
    
    pop_w['Date'] = sorted(fechas_semanales) * len(pop_s)
    
    pop_w = pop_w.reset_index(drop=True)
    pop_w = pop_w.reindex(columns=['DPMP', 'Date'] + list(pop.columns[2:]))
    pop_w = pop_w.ffill()
    pop_W.append(pop_w)
pop_w = pd.concat(pop_W)
pop_w

,DPMP,Date,Hombres_0_19,Hombres_20_39,Hombres_40_59,Hombres_60_79,Hombres_80+,Mujeres_0_19,Mujeres_20_39,Mujeres_40_59,Mujeres_60_79,Mujeres_80+,Total_0_19,Total_20_39,Total_40_59,Total_60_79,Total_80+,Total_Hombres,Total_Mujeres,Total_General
0,5001,2007-01-07,343775,313794,227004,83314,11077,341909,356136,293295,110307,21160,685684,669930,520299,193621,32237,978964,1122807,2101771
1,5001,2007-01-14,343775,313794,227004,83314,11077,341909,356136,293295,110307,21160,685684,669930,520299,193621,32237,978964,1122807,2101771
2,5001,2007-01-21,343775,313794,227004,83314,11077,341909,356136,293295,110307,21160,685684,669930,520299,193621,32237,978964,1122807,2101771
3,5001,2007-01-28,343775,313794,227004,83314,11077,341909,356136,293295,110307,21160,685684,669930,520299,193621,32237,978964,1122807,2101771
4,5001,2007-02-04,343775,313794,227004,83314,11077,341909,356136,293295,110307,21160,685684,669930,520299,193621,32237,978964,1122807,2101771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58339,99773,2021-11-28,21140,12287,6445,2242,258,19885,10733,5156,1653,185,41025,23020,11601,3895,443,42372,37612,79984
58340,99773,2021-12-05,21140,12287,6445,2242,258,19885,10733,5156,1653,185,41025,23020,11601,3895,443,42372,37612,79984
58341,99773,2021-12-12,21140,12287,6445,2242,258,19885,10733,5156,1653,185,41025,23020,11601,3895,443,42372,37612,79984
58342,99773,2021-12-19,21140,12287,6445,2242,258,19885,10733,5156,1653,185,41025,23020,11601,3895,443,42372,37612,79984


In [14]:
correl = pd.read_csv('../Data/mpios_correlation.csv')
correl

,Unnamed: 0,Mpio,coef_1,coef_2,coef_3,coef_4,coef_5,Año
0,0,5001,0.996612,0.888604,0.0,0.917858,0.0,2007
1,1,5002,0.000000,0.000000,0.0,0.000000,0.0,2007
2,2,5004,0.000000,0.000000,0.0,0.000000,0.0,2007
3,3,5021,0.000000,0.000000,0.0,0.000000,0.0,2007
4,4,5030,0.000000,0.000000,0.0,0.000000,0.0,2007
...,...,...,...,...,...,...,...,...
16825,16825,15600,0.000000,0.000000,0.0,0.000000,0.0,2021
16826,16826,25779,0.000000,0.000000,0.0,0.000000,0.0,2021
16827,16827,15822,0.000000,0.000000,0.0,0.000000,0.0,2021
16828,16828,97777,0.000000,0.000000,0.0,0.000000,0.0,2021


In [16]:
correl

,Mpio,coef_1,coef_2,coef_3,coef_4,coef_5,Año
0,5001,0.996612,0.888604,0.0,0.917858,0.0,2007
1,5002,0.000000,0.000000,0.0,0.000000,0.0,2007
2,5004,0.000000,0.000000,0.0,0.000000,0.0,2007
3,5021,0.000000,0.000000,0.0,0.000000,0.0,2007
4,5030,0.000000,0.000000,0.0,0.000000,0.0,2007
...,...,...,...,...,...,...,...
16825,15600,0.000000,0.000000,0.0,0.000000,0.0,2021
16826,25779,0.000000,0.000000,0.0,0.000000,0.0,2021
16827,15822,0.000000,0.000000,0.0,0.000000,0.0,2021
16828,97777,0.000000,0.000000,0.0,0.000000,0.0,2021


In [20]:
columns = ['Mpio', 'coef_1', 'coef_2', 'coef_3', 'coef_4', 'coef_5', 'Año']
correl = correl[columns]

correl_W = []
for año in [2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021]:

    fechas_semanales = pd.date_range(start=str(año)+'-01-01', end=str(año)+'-12-31', freq='W')
    
    df_final = pd.DataFrame()

    dptos = correl['Mpio'].unique()

    num_repeticiones = len(fechas_semanales)
    correl_s = correl[correl['Año']==año]
    correl_w = correl_s.loc[correl_s.index.repeat(num_repeticiones)]
    
    correl_w['Date'] = sorted(fechas_semanales) * len(pop_s)
    
    correl_w = correl_w.reset_index(drop=True)
    correl_w = correl_w.reindex(columns=['Mpio', 'Date'] + list(correl.columns[1:6]))
#     correl_w = correl_w.ffill()
    correl_W.append(correl_w)
correl_W = pd.concat(correl_W)
correl_W

,Mpio,Date,coef_1,coef_2,coef_3,coef_4,coef_5
0,5001,2007-01-07,0.996612,0.888604,0.0,0.917858,0.0
1,5001,2007-01-14,0.996612,0.888604,0.0,0.917858,0.0
2,5001,2007-01-21,0.996612,0.888604,0.0,0.917858,0.0
3,5001,2007-01-28,0.996612,0.888604,0.0,0.917858,0.0
4,5001,2007-02-04,0.996612,0.888604,0.0,0.917858,0.0
...,...,...,...,...,...,...,...
58339,15837,2021-11-28,0.000000,0.000000,0.0,0.000000,0.0
58340,15837,2021-12-05,0.000000,0.000000,0.0,0.000000,0.0
58341,15837,2021-12-12,0.000000,0.000000,0.0,0.000000,0.0
58342,15837,2021-12-19,0.000000,0.000000,0.0,0.000000,0.0


In [ ]:
dane = pd.read_excel('pobreza_mpios.xlsx')
dane

In [ ]:
dane_W = []
for año in [2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021]:

    fechas_semanales = pd.date_range(start=str(año)+'-01-01', end=str(año)+'-12-31', freq='W')

    # 2. Inicializa un DataFrame vacío para almacenar los datos
    #    de todas las características para cada departamento.
    df_final = pd.DataFrame()

    dptos = dane['Cod_Mpio'].unique()

    num_repeticiones = len(fechas_semanales)
    dane_s = dane[dane['Año']==año]
    dane_w = dane_s.loc[dane_s.index.repeat(num_repeticiones)]

    # Asignar las fechas a las filas replicadas
    dane_w['Date'] = sorted(fechas_semanales) * len(dane_s)

    # Repetir los valores de las 29 características para cada fecha
    dane_w = dane_w.reset_index(drop=True)
    dane_w = dane_w.reindex(columns=['Cod_Mpio', 'Municipio', 'Date'] + list(dane.columns[3:]))
    dane_w = dane_w.ffill()
    dane_W.append(dane_w)
dane_w = pd.concat(dane_W)
dane_w

In [22]:
spei = pd.read_csv('../Data/SPEI_mpios.csv')
spei

,id,max_SPEI,mean_SPEI,median_SPEI,min_SPEI,std_SPEI,mpio
0,2007-01-01,-0.262771,-0.411996,-0.383579,-0.533585,-0.383579,18001
1,2007-02-01,-2.045366,-2.105320,-2.093575,-2.189453,-2.093575,18001
2,2007-03-01,0.595827,0.241394,0.202247,-0.221765,0.202247,18001
3,2007-04-01,1.885987,1.692926,1.741783,1.452462,1.741783,18001
4,2007-05-01,0.249808,-0.200942,-0.233363,-0.652896,-0.233363,18001
...,...,...,...,...,...,...,...
201955,2021-08-01,1.908579,1.908579,1.908579,1.908579,1.908579,5380
201956,2021-09-01,0.224510,0.224510,0.224510,0.224510,0.224510,5380
201957,2021-10-01,-0.028427,-0.028427,-0.028427,-0.028427,-0.028427,5380
201958,2021-11-01,1.275174,1.275174,1.275174,1.275174,1.275174,5380


In [23]:
fechas_semanales = pd.date_range(start='2007-01-01', end='2021-12-31', freq='W')
spei['id'] = pd.to_datetime(spei['id'])

mpios = spei['mpio'].unique()
columnas_repetir = ['max_SPEI', 'mean_SPEI', 'median_SPEI', 'min_SPEI', 'std_SPEI']

spei_final = pd.DataFrame()

for mpio in mpios:
    spei_mpio = spei[spei['mpio'] == mpio].copy()
    spei_mpio.set_index('id', inplace=True)
    spei_mpio_semanal = spei_mpio.reindex(fechas_semanales, method='ffill')
    spei_mpio_semanal['mpio'] = mpio
    spei_final = pd.concat([spei_final, spei_mpio_semanal])

spei_final.reset_index(inplace=True)

spei_final = spei_final.sort_values('mpio').fillna(method = 'ffill')
spei_final = spei_final.sort_values('index')

print(spei_final)

            index  max_SPEI  mean_SPEI  median_SPEI  min_SPEI  std_SPEI   mpio
269008 2007-01-07 -0.188684  -0.254635    -0.306570 -0.306570 -0.306570   5148
77418  2007-01-07 -0.084384  -0.112711    -0.098973 -0.168471 -0.098973  76109
687378 2007-01-07 -0.832972  -0.841605    -0.832972 -0.936926 -0.832972  54313
869584 2007-01-07  0.016245  -0.122169    -0.242817 -0.242817 -0.242817  25754
614652 2007-01-07  0.003605   0.003605     0.003605  0.003605  0.003605  25288
...           ...       ...        ...          ...       ...       ...    ...
724913 2021-12-26 -0.892766  -1.077539    -1.049932 -1.238770 -1.049932  50573
28151  2021-12-26 -0.674592  -0.674592    -0.674592 -0.674592 -0.674592  19513
330003 2021-12-26 -0.014763  -0.094027    -0.137256 -0.238079 -0.137256  27372
733515 2021-12-26 -1.232468  -1.405608    -1.380245 -1.884669 -1.380245  50568
508299 2021-12-26 -1.048535  -1.048535    -1.048535 -1.048535 -1.048535  15131

[877404 rows x 7 columns]


In [24]:
dataset = pd.merge(dengue, climate, left_on=['Date', 'Mpio'], right_on=['id', 'mpio'], how='outer')#.fillna(0)
dataset[['Dengue', 'Dengue_grave', 'Mortalidad_por_dengue']] = dataset[['Dengue', 'Dengue_grave', 'Mortalidad_por_dengue']].fillna(0.0)
dataset = dataset.drop(columns=["Date", 'Mpio']).rename(columns = {'id':'Date','mpio':'Mpio'})
dataset = pd.merge(dataset, rain, left_on=['Date', 'Mpio'], right_on=['id', 'mpio'], how='outer')
dataset = dataset.drop(columns=["id", 'mpio'])
dataset = pd.merge(dataset, pop_w, left_on=['Date', 'Mpio'], right_on=['Date', 'DPMP'], how='outer')
dataset = dataset.drop(columns=['DPMP'])
dataset = pd.merge(dataset, correl_W, left_on=['Date', 'Mpio'], right_on=['Date', 'Mpio'], how='outer')
# # dataset = pd.merge(dataset, dane_w, left_on=['Date', 'Mpio'], right_on=['Date', 'Cod_Mpio'], how='outer')
# # dataset = dataset.drop(columns=['Cod_Mpio', 'Municipio'])
dataset = pd.merge(dataset, spei_final, left_on=['Date', 'Mpio'], right_on=['index', 'mpio'], how='outer')
dataset = dataset.drop(columns=['mpio', 'index'])
dataset = pd.merge(dataset, spatial_replicado, left_on=['Date', 'Mpio'], right_on=['Date', 'MPIO_CDPMP'], how='outer')
dataset = dataset.drop(columns=['MPIO_CDPMP'])
dataset.dropna(inplace=True)
dataset = dataset.set_index(['Date', 'Mpio']).reset_index()
dataset

,Date,Mpio,Dengue,Dengue_grave,Mortalidad_por_dengue,max_temperature,mean_temperature,min_temperature,mean_rain,Hombres_0_19,...,coef_4,coef_5,max_SPEI,mean_SPEI,median_SPEI,min_SPEI,std_SPEI,AREA,LATITUD,LONGITUD
0,2007-01-07,5001,0.0,0.0,0.0,32.47,22.808215,11.03,4.057847,343775,...,0.917858,0.0,-0.132057,-0.138179,-0.132057,-0.188684,-0.132057,3.748306e+08,6.257590,-75.611031
1,2007-01-07,5002,0.0,0.0,0.0,31.95,23.834452,14.97,11.162859,4502,...,0.000000,0.0,-0.148218,-0.274834,-0.306570,-0.306570,-0.306570,5.071341e+08,5.803728,-75.438474
2,2007-01-07,5004,0.0,0.0,0.0,24.69,17.786313,8.03,6.745960,537,...,0.000000,0.0,0.022054,0.022054,0.022054,0.022054,0.022054,2.969560e+08,6.627569,-76.085978
3,2007-01-07,5021,0.0,0.0,0.0,26.47,20.608759,14.59,12.821658,986,...,0.000000,0.0,-0.188684,-0.203064,-0.188684,-0.442003,-0.188684,1.289322e+08,6.365534,-75.090597
4,2007-01-07,5030,0.0,0.0,0.0,28.35,20.231379,14.53,7.179329,5417,...,0.000000,0.0,-0.132057,-0.134742,-0.132057,-0.148218,-0.132057,8.413248e+07,6.032922,-75.708003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877399,2021-12-26,15272,0.0,0.0,0.0,29.15,22.712172,16.39,5.042560,1017,...,0.000000,0.0,-1.043489,-1.081897,-1.043489,-1.153882,-1.043489,1.093406e+08,5.673677,-73.019834
877400,2021-12-26,52354,0.0,0.0,0.0,23.85,16.965397,9.37,17.779865,886,...,0.000000,0.0,-0.346310,-0.358698,-0.346310,-0.374052,-0.346310,8.172986e+07,1.072877,-77.501509
877401,2021-12-26,15276,0.0,0.0,0.0,26.11,21.568825,16.15,5.398320,452,...,0.000000,0.0,-1.153882,-1.153882,-1.153882,-1.153882,-1.153882,8.812846e+07,5.858992,-72.924998
877402,2021-12-26,52323,0.0,0.0,0.0,21.57,12.931046,6.97,14.978008,865,...,0.000000,0.0,0.045646,0.045646,0.045646,0.045646,0.045646,2.996187e+07,0.928643,-77.582619


In [26]:
dataset.to_csv('../Data/finaldata_mpios_corr5.csv', index=False)